# Model

> Scripts to train the models

In [ ]:
#| default_exp model

In [ ]:
#| export
#| hide
from keras.optimizers import Adam, SGD

2024-05-19 09:46:57.120042: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [ ]:
#| export
def get_optimizer(optimizer_config):
    name = optimizer_config['name'].lower()
    if name == 'adam':
        return Adam(learning_rate=optimizer_config.get('learning_rate', 0.001))
    elif name == 'sgd':
        return SGD(learning_rate=optimizer_config.get('learning_rate', 0.01))
    # Add additional optimizers as needed
    raise ValueError("Unsupported optimizer: {}".format(optimizer_config['name']))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()